# Retrieval script for pulling data out of the testing or production ESIP Data Management Training Clearinghouse

API Documentation: [https://docs.google.com/document/d/15czKFzgWNw2f-nQRZ9mMU3Elh2eETb2eBKx2XNvRGcE/edit](https://docs.google.com/document/d/15czKFzgWNw2f-nQRZ9mMU3Elh2eETb2eBKx2XNvRGcE/edit)

In [1]:
import json
from sys import exit
import requests
import getpass
import pandas as pd
from bs4 import BeautifulSoup
import pickle

In [19]:
def getIDs():
    """Returns data from either the testing or production Data Management Training Clearinghouse system as a JSON string. 
    
    Keyword arguments:
    testing     -- True (default) | False value that toggles the function between testing and production modes. If testing the request will be sent to the testing server with default credentials. If not testing the request will be sent to the production server and the user will be asked to provide a username and password when executing the function. 
    """
    
    print("========= authentication =========")
    base_url: str = 'http://dmtclearinghouse.esipfed.org/api/dmt/search'
    login_data = {
        'username': getpass.getpass("Enter your username - the input will be hidden"),
        'password': getpass.getpass("Enger your password - the input will be hidden")}
    print("==================================\n\n")
    
    
    headerString = {'Content-type': 'application/json'}
    
    r = requests.post(url = base_url+ '/user/login', headers = headerString, data = json.dumps(login_data) )
    
    #print('loginStatusCode: '+str(r.status_code))

    #print('loginText: \n'+r.text)
    #print('loginCookies: \n'+str(r.cookies) )

    cookieObj=r.cookies
    #print()
    
    r = requests.post(url = base_url+ '/user/token.json', cookies=cookieObj, headers = headerString, data = json.dumps(login_data) )
    #print( 'tokenStatusCode" '+str(r.status_code))

    #print( 'tokenText" '+r.text)

    tokenTxt=r.text.replace('{','').replace('}','').replace("token",'').replace('"','').replace(':','')
    #print( 'cleanedUpToken: '+tokenTxt)
    #print()
    
    #headerString = {'X-CSRF-TOKEN' : tokenTxt}

    get_query = requests.get(base_url+'/search_api/search_index_dmt_clearinghouse.json', cookies=cookieObj, headers = headerString)
    #print( 'getQueryStatusCode: '+str(get_query.status_code) )
    #print()

    #print( 'getQueryText: \n'+get_query.text)
    #print()
    
    return(json.loads(get_query.text)['result'].keys())

def getData(queryString='', testing=True):
    """Returns data from either the testing or production Data Management Training Clearinghouse system as a JSON string. 
    
    Keyword arguments:
    queryString -- the query string that should be appended to the base request URL. Defaults to ''
    testing     -- True (default) | False value that toggles the function between testing and production modes. If testing the request will be sent to the testing server with default credentials. If not testing the request will be sent to the production server and the user will be asked to provide a username and password when executing the function. 
    """
    
    print("========= authentication =========")
    if testing:
        base_url: str = 'http://test.dmtclearinghouse.esipfed.bluedotapps.org/api/dmt/search'
        login_data = {
            'username': 'api.persona',
            'password': 'api.persona'}
    else:
        base_url: str = 'http://dmtclearinghouse.esipfed.org/api/dmt/search'
        login_data = {
            'username': getpass.getpass("Enter your username - the input will be hidden"),
            'password': getpass.getpass("Enger your password - the input will be hidden")}
    print("==================================\n\n")
    
    
    headerString = {'Content-type': 'application/json'}
    
    r = requests.post(url = base_url+ '/user/login', headers = headerString, data = json.dumps(login_data) )
    
    #print('loginStatusCode: '+str(r.status_code))

    #print('loginText: \n'+r.text)
    #print('loginCookies: \n'+str(r.cookies) )

    cookieObj=r.cookies
    #print()
    
    r = requests.post(url = base_url+ '/user/token.json', cookies=cookieObj, headers = headerString, data = json.dumps(login_data) )
    #print( 'tokenStatusCode" '+str(r.status_code))

    #print( 'tokenText" '+r.text)

    tokenTxt=r.text.replace('{','').replace('}','').replace("token",'').replace('"','').replace(':','')
    #print( 'cleanedUpToken: '+tokenTxt)
    #print()
    
    #headerString = {'X-CSRF-TOKEN' : tokenTxt}

    get_query = requests.get(base_url+'/search_api/search_index_dmt_clearinghouse.json'+queryString, cookies=cookieObj, headers = headerString)
    #print( 'getQueryStatusCode: '+str(get_query.status_code) )
    #print()

    #print( 'getQueryText: \n'+get_query.text)
    #print()
    
    return(json.loads(get_query.text))

def json2list(dmtc_json):
    json_sub = dmtc_json['result']
    results = []
    i = 1
    #print(json_sub)
    for item in json_sub.values():
        result_dict = {}
        result_dict['id'] = item['id']
        print(str(i),": ", item['id'])
        
        fields = item['fields']
        #print(fields,"\n-----------\n")
        for key, value in fields.items():
            if isinstance(value, dict):
                #print(key,": dvalue ",value['und'])
                result_dict[key] = list(value['und'][0].values())
            elif isinstance(value, list):
                #print(key,": lvalue",value)
                result_dict[key] = value
            else:
                result_dict[key] = None
            
        #frames.append(pd.DataFrame.from_dict(result_dict, orient="columns"))
        results.append(result_dict)
        i += 1
        #print(result_dict)
        #print(item)
        #print("\n==========================\n")
    
    
    # result_df = pd.concat(frames)
    return(results)
    
def scrapeDMTC(fields, testing=True):
    # Web Scraping approach to pulling values from core nodes and associated reference tables. 
    baseURL = "http://dmtclearinghouse.esipfed.org/node/"
    
    testCount = 6
    i = 1
    results = []
    for nodeID in getIDs():
        node2get = baseURL+nodeID
        if testing and i > testCount:
            print("skipping: ", node2get)
        else:
            print(str(i),". processing: ", node2get)
            r = requests.get(node2get)
            soup = BeautifulSoup(r.text)
            result = {}
            result['id'] = nodeID
            for field in fields:
                content = []
                try:
                    if field == "field-name-title":
                        content = [soup.find("div", class_=field).get_text()]
                    elif field == "field-name-field-lr-url":
                        links = soup.find("div", class_="field-name-field-lr-url").find_all("a")
                        for link in links:
                            content.append(link.get_text())
                    elif field == "field-name-field-lr-date-published":
                        items = soup.find_all("span", class_="date-display-single")
                        for item in items:
                            content.append(item['content'])
                    elif field == "field-name-field-lr-contributor-people":
                        items = soup.find("div", class_=field).find_all("div", class_="field-item")
                        for item in items:
                            nameItems = item.find_all("div", class_="field-item")
                            nameList = []
                            for nameItem in nameItems:
                                nameList.append(nameItem.get_text())
                            if len(nameList) > 0:
                                content.append(nameList)
                    else:
                        items = soup.find("div", class_=field).find_all("div", class_="field-item")
                        for item in items:
                            content.append(item.get_text())
                    
                except:
                    content = None
                #print()
                #print(field,": ", content)
                result[field] = content
            #print(result)
            results.append(result)
            
        #print()
        #print("======================")
        i += 1
    return(results)    

In [20]:
# Scraping of the DMTC website for resolved metadata values that
# integrate core and related table contents. 
scrapeFields = [
        "field-name-title",
        "field-name-field-lr-url",
        "field-name-field-lr-abstract",
        "field-name-field-lr-author-people",
        "field-name-field-lr-license",
        "field-name-field-lr-usage-rights",
        "field-name-field-lr-payment-required",
        "field-name-field-lr-citation",
        "field-name-field-lr-language-primary",
        "field-name-field-lr-languages-secondary",
        "field-name-field-lr-keywords",
        "field-name-field-lr-subject",
        "field-name-field-lr-date-published",
        "field-name-field-lr-locator-id",
        "field-name-field-lr-locator-type",
        "field-name-field-lr-publisher",
        "field-name-field-lr-version",
        "field-name-field-lr-media-type",
        "field-name-field-lr-contributor-people",
        "field-name-field-lr-contributor-orgs",
        "field-name-field-lr-contact-people",
        "field-name-field-lr-contact-organizations",
        "field-name-field-lr-ed-purpose",
        "field-name-field-lr-type",
        "field-name-field-lr-ed-audience",
        "field-name-field-lr-completion-time",
        "field-name-field-lr-ed-framework",
        "field-name-field-lr-ed-framework-node-esip",
        "field-name-field-lr-ed-framework-node-usgs",
        "field-name-field-lr-ed-framework-node-data1",
        "field-name-field-lr-framework-node-fair",
        "field-name-field-dmt-submitter-name",
        "field-name-field-submission-contact-email-a",
        "field-name-field-lr-access-features"
    ]

scrapeResults = scrapeDMTC(scrapeFields, testing=False)

# API Parameter list for retrieving supplementatal data through the API
# that don't get exposed through the web interface
parameterList = [
    "field_dmt_submitter_name",
    "field_submission_contact_email_a",
    "field_lr_usage_rights",
    "field_lr_languages_secondary",
    "field_lr_contributor_orgs",
    "field_lr_access_features"
]
parameters = "?fields[]="+"&fields[]=".join(parameterList)
#print(parameters)
#getResult = getData(testing=False, queryString = parameters)

print()
suppResults = json2list(getData(testing=False, queryString = parameters))


========= authentication =========
Enter your username - the input will be hidden········
Enger your password - the input will be hidden········


1 . processing:  http://dmtclearinghouse.esipfed.org/node/9410
2 . processing:  http://dmtclearinghouse.esipfed.org/node/9412
3 . processing:  http://dmtclearinghouse.esipfed.org/node/9413
4 . processing:  http://dmtclearinghouse.esipfed.org/node/9415
5 . processing:  http://dmtclearinghouse.esipfed.org/node/9570
6 . processing:  http://dmtclearinghouse.esipfed.org/node/9571
7 . processing:  http://dmtclearinghouse.esipfed.org/node/9572
8 . processing:  http://dmtclearinghouse.esipfed.org/node/9573
9 . processing:  http://dmtclearinghouse.esipfed.org/node/9574
10 . processing:  http://dmtclearinghouse.esipfed.org/node/9576
11 . processing:  http://dmtclearinghouse.esipfed.org/node/9577
12 . processing:  http://dmtclearinghouse.esipfed.org/node/9578
13 . processing:  http://dmtclearinghouse.esipfed.org/node/9579
14 . processing:  http://dmtcl

126 . processing:  http://dmtclearinghouse.esipfed.org/node/10008
127 . processing:  http://dmtclearinghouse.esipfed.org/node/10017
128 . processing:  http://dmtclearinghouse.esipfed.org/node/10018
129 . processing:  http://dmtclearinghouse.esipfed.org/node/10020
130 . processing:  http://dmtclearinghouse.esipfed.org/node/10021
131 . processing:  http://dmtclearinghouse.esipfed.org/node/10022
132 . processing:  http://dmtclearinghouse.esipfed.org/node/10023
133 . processing:  http://dmtclearinghouse.esipfed.org/node/10024
134 . processing:  http://dmtclearinghouse.esipfed.org/node/10025
135 . processing:  http://dmtclearinghouse.esipfed.org/node/10029
136 . processing:  http://dmtclearinghouse.esipfed.org/node/10030
137 . processing:  http://dmtclearinghouse.esipfed.org/node/10033
138 . processing:  http://dmtclearinghouse.esipfed.org/node/10035
139 . processing:  http://dmtclearinghouse.esipfed.org/node/10040
140 . processing:  http://dmtclearinghouse.esipfed.org/node/9770
141 . proce

251 . processing:  http://dmtclearinghouse.esipfed.org/node/9939
252 . processing:  http://dmtclearinghouse.esipfed.org/node/10255
253 . processing:  http://dmtclearinghouse.esipfed.org/node/10109
254 . processing:  http://dmtclearinghouse.esipfed.org/node/10202
255 . processing:  http://dmtclearinghouse.esipfed.org/node/10121
256 . processing:  http://dmtclearinghouse.esipfed.org/node/10120
257 . processing:  http://dmtclearinghouse.esipfed.org/node/10112
258 . processing:  http://dmtclearinghouse.esipfed.org/node/10180
259 . processing:  http://dmtclearinghouse.esipfed.org/node/10188
260 . processing:  http://dmtclearinghouse.esipfed.org/node/10116
261 . processing:  http://dmtclearinghouse.esipfed.org/node/9907
262 . processing:  http://dmtclearinghouse.esipfed.org/node/10346
263 . processing:  http://dmtclearinghouse.esipfed.org/node/9976
264 . processing:  http://dmtclearinghouse.esipfed.org/node/10345
265 . processing:  http://dmtclearinghouse.esipfed.org/node/10260
266 . process

In [55]:
# Build dataframes
scrapeDF = pd.DataFrame.from_dict(scrapeResults)
scrapeDF['id'] = scrapeDF['id'].astype(str)
scrapeDF = scrapeDF.set_index('id')
suppDF = pd.DataFrame.from_dict(suppResults)
suppDF['id'] = suppDF['id'].astype(str)
suppDF = suppDF.set_index('id')
mergedDF = pd.merge(scrapeDF, suppDF, on='id').sort_index()

In [56]:
mergedDF

,field-name-field-dmt-submitter-name,field-name-field-lr-abstract,field-name-field-lr-access-features,field-name-field-lr-author-people,field-name-field-lr-citation,field-name-field-lr-completion-time,field-name-field-lr-contact-organizations,field-name-field-lr-contact-people,field-name-field-lr-contributor-orgs,field-name-field-lr-contributor-people,...,field-name-field-lr-usage-rights,field-name-field-lr-version,field-name-field-submission-contact-email-a,field-name-title,field_dmt_submitter_name,field_lr_access_features,field_lr_contributor_orgs,field_lr_languages_secondary,field_lr_usage_rights,field_submission_contact_email_a
id,,,,,,,,,,,,,,,,,,,,,
10000,None,[The first webinar organized by the GODAN (Glo...,None,"[André Laperrière, Suchith Anand, Isaura Lo...",None,[Up to 1 hour],None,None,None,None,...,None,None,None,[GODAN Working Group on Capacity Development],"[Nancy H., None, Nancy H.]",[],[],[],[],[nhoebel@kmotifs.com]
10001,None,[The course provides an introduction to the Oc...,None,"[Dairo Escobar, Venus Rocha, Julio Bohorquez, ...",None,[More than 1 hour (but less than 1 day)],None,None,None,None,...,None,None,None,[Administración de Datos Biogeográficos Marino...,"[Abby Pennington, None, Abby Pennington]",[],[],[],[],[abbypenn93@gmail.com]
10002,None,[The course provided an introduction to the Oc...,None,[Ward Appeltans],None,[More than 1 hour (but less than 1 day)],None,None,None,None,...,None,None,None,[Marine Biogeographic Data Management (Contrib...,"[Abby Pennington, None, Abby Pennington]",[],[],[],[],[abbypenn93@gmail.com]
10003,None,[This introductory chapter features a brief in...,None,"[Ulf Jakobsson, Ricarda Braukmann, Malin Lundg...",None,[More than 1 Day (but less than 1 week)],None,None,None,None,...,None,None,None,"[Plan, a chapter of the CESSDA Expert Tour Gui...","[Nancy Hoebelheinrich, None, Nancy Hoebelheinr...",[],[],[],[],[nhoebel@kmotifs.com]
10004,None,"[In this chapter, we provide you with tips and...",None,"[Jindrich Krejčí, Johana Chylikova, Katya F...",None,[More than 1 Day (but less than 1 week)],None,None,None,None,...,None,None,None,"[Organise & Document, a chapter of the CESSDA ...","[Nancy Hoebelheinrich, None, Nancy Hoebelheinr...",[],[],[],[],[nhoebel@kmotifs.com]
10005,None,[Marine information managers are increasingly ...,None,"[Daryl Superio, Greg Reed]",None,[More than 1 hour (but less than 1 day)],None,None,[\n\nName: Australian AidType: Funding and spo...,None,...,None,None,None,[Research Data Management ],"[Abby Pennington, None, Abby Pennington]",[],[314],[],[],[abbypenn93@gmail.com]
10006,None,[In this chapter we focus on data operations n...,None,"[Jindrich Krejčí, Johana Chylikova]",None,[More than 1 Day (but less than 1 week)],None,None,None,None,...,None,None,None,"[Process, a chapter of the CESSDA Expert Tour ...","[Nancy Hoebelheinrich, None, Nancy Hoebelheinr...",[],[],[],[],[nhoebel@kmotifs.com]
10007,None,"[The data that you collect, organise, prepare,...",None,[Jonas Recker],None,[More than 1 Day (but less than 1 week)],None,None,None,None,...,None,None,None,"[Store, a chapter of the CESSDA Expert Tour on...","[Nancy Hoebelheinrich, None, Nancy Hoebelheinr...",[],[],[],[],[nhoebel@kmotifs.com]
10008,None,[There is a growing awareness of the importanc...,None,"[Helena Cousijn, Tim Clarke]",None,[Up to 1 hour],None,None,None,None,...,None,None,None,[Making Research Data Available],"[Abby Pennington, None, Abby Pennington]",[],[],[],[],[abbypenn93@gmail.com]


In [57]:
mergedDF.to_excel("dmtc_data.xlsx")